
# Análisis de Secuencias Nucleotídicas de Integronas de Shewanella

Este cuaderno realiza la descarga y análisis de secuencias nucleotídicas de integronas de Shewanella utilizando Biopython.




Este código descarga el dataset de secuencias de integrasas de Shewanella desde el NCBI. Luego, realiza un alineamiento múltiple de las secuencias para identificar las regiones conservadas. A continuación, selecciona un objetivo blanco para las guías CRISPR/Cas9. Finalmente, diseña las guías CRISPR/Cas9 y realiza la transformación de las bacterias de Shewanella que albergan integrones.

Para verificar la edición mediante secuenciación, se pueden diseñar primers específicos para la región editada. El código anterior muestra cómo diseñar primers para la región editada del gen de la integrasa.



**1. Breve introducción**

Los integrones son plataformas genéticas que se encuentran comúnmente en bacterias. Su función principal es la captura y la expresión de genes, los cuales están especialmente asociados con la resistencia a antibióticos. La tecnología CRISPR ha demostrado ser una herramienta poderosa para la edición genética precisa.

**2. Hipótesis de Trabajo y Resultados Esperados**

La hipótesis del proyecto es que el uso de CRISPR/Cas9 específicamente dirigido al gen de la integrasa en integrones de bacterias del género Shewanella puede resultar en la interrupción o desactivación de la función de recombinación de ADN de la integrasa. Se hipotetiza que al modificar el gen de la integrasa, se reducirá la capacidad de los integrones para capturar y expresar genes asociados con la resistencia a antibióticos en las bacterias de este género.

Los resultados esperados del proyecto son:

* Demostrar la capacidad de la tecnología CRISPR/Cas9 para dirigirse con precisión al gen de la integrasa de los integrones encontrados en Shewanella, evidenciando la capacidad de editar este gen de manera específica.
* Observar una disminución en la capacidad de los integrones modificados para realizar la recombinación del ADN, demostrando así la influencia de la edición genética en la función de la integrasa.
* Observar una reducción en la expresión de genes asociados con la resistencia a antibióticos en las bacterias de Shewanella portadoras de integrones editados.


In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq

# Supongamos que ya tienes los archivos FASTA descargados

# Función para modelar la edición de integrones en Shewanella usando CRISPR/Cas9
def edit_integrons(accession_numbers):
    for accession_number in accession_numbers:
        # Leer la secuencia de los integrones
        with open(accession_number + ".fasta", "r") as file:
            integron_sequence = SeqIO.read(file, "fasta")

        # Diseñar las guías CRISPR/Cas9
        guides = design_guides(integron_sequence.seq)

        # Transformar las bacterias de Shewanella con el sistema CRISPR/Cas9
        transform_bacteria(guides)

        # Verificar la edición mediante secuenciación
        verify_editing(accession_number)

# Función para diseñar las guías CRISPR/Cas9 (implementación conceptual)

def design_guides(sequence):
    # Identificar regiones conservadas y posibles sitios de corte
    conserved_regions = find_conserved_regions(sequence)
    targets = evaluate_target_sites(conserved_regions)
    guides = design_guides_from_targets(targets)
    return guides

# Función para encontrar regiones conservadas (implementación conceptual)
def find_conserved_regions(sequence):
    # Aquí se podría usar un enfoque bioinformático para identificar regiones conservadas
    return ["region_conservada_1", "region_conservada_2"]

# Función para evaluar posibles sitios de corte (implementación conceptual)
def evaluate_target_sites(conserved_regions):
    # Aquí se evaluarían las regiones conservadas para identificar posibles sitios de corte CRISPR
    return ["sitio_corte_1", "sitio_corte_2"]

# Función para diseñar guías a partir de objetivos (implementación conceptual)
def design_guides_from_targets(targets):
    # Aquí se diseñarían las guías CRISPR basadas en los sitios de corte identificados
    return ["guía_1", "guía_2"]

# Función para transformar bacterias (implementación conceptual)
def transform_bacteria(guides):
    # Aquí se implementaría el proceso de transformación de Shewanella con CRISPR/Cas9
    print("Transformando bacterias con guías CRISPR/Cas9")

# Función para verificar la edición (implementación conceptual)
def verify_editing(accession_number):
    # Aquí se diseñarían primers y se realizaría secuenciación para verificar la edición genética
    print(f"Verificando la edición del gen en {accession_number}")

# Ejecución de las funciones
edit_integrons(accession_numbers)


Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_016848.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_027179.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_043643.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_048237.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_052996.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_053976.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_058644.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_059384.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_062436.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en NC_064046.1
Transformando bacterias con guías CRISPR/Cas9
Verificando la edición del gen en 

In [ ]:
#
def transform_bacteria(guides, sequence):
    mutations = []
    for guide in guides:
        if random.random() < 0.5:  # 50% de posibilidades de una edición exitosa
            start = sequence.find(guide)
            if start != -1:
                end = start + len(guide)
                mutation_type = random.choice(["deletion", "insertion", "substitution"])
                if mutation_type == "deletion":
                    sequence = sequence[:start] + sequence[end:]
                    mutations.append((mutation_type, start, sequence[start:end], ""))
                elif mutation_type == "insertion":
                    insertion_seq = "ATCG"
                    sequence = sequence[:start] + insertion_seq + sequence[start:]
                    mutations.append((mutation_type, start, "", insertion_seq))
                elif mutation_type == "substitution":
                    substitution_seq = "ATCG"
                    sequence = sequence[:start] + substitution_seq + sequence[end:]
                    mutations.append((mutation_type, start, sequence[start:end], substitution_seq))
    return sequence, mutations

def verify_editing(accession_number, original_sequence, edited_sequence, mutations):
    if original_sequence != edited_sequence:
        print(f"Edición genética verificada para {accession_number}")
        print("Detalle de las mutaciones:")
        for mutation in mutations:
            mutation_type, position, original, new = mutation
            print(f"{mutation_type.capitalize()} en posición {position}: Original '{original}', Nuevo '{new}'")

        # Representación gráfica de las ediciones
        print("\nRepresentación Gráfica de las Ediciones:")
        graphical_representation(original_sequence, edited_sequence, mutations)
        return edited_sequence
    else:
        print(f"No se detectó edición genética para {accession_number}")
        return None

def graphical_representation(original_seq, edited_seq, mutations):
    print("Original: " + original_seq[:50] + "..." + original_seq[-50:])
    print("Editada : " + edited_seq[:50] + "..." + edited_seq[-50:])
    for mutation in mutations:
        mutation_type, position, original, new = mutation
        print(f"{'^' * len(original)} {mutation_type.capitalize()} en posición {position}")

# Ejemplo de uso
accession_number = "NC_016848.1"
with open(accession_number + ".fasta", "r") as file:
    original_sequence = SeqIO.read(file, "fasta").seq

guides = ["GTTACG", "CGTACG"]  # Ejemplo de guías CRISPR
edited_sequence, mutations = transform_bacteria(guides, original_sequence)

edited_seq_record = verify_editing(accession_number, original_sequence, edited_sequence, mutations)
if edited_seq_record:
    SeqIO.write(SeqRecord(Seq(edited_seq_record), id=accession_number), f"{accession_number}_edited.fasta", "fasta")

NameError: ignored

In [ ]:
import random
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import matplotlib.pyplot as plt

# Función para transformar bacterias simulando ediciones CRISPR/Cas9
def transform_bacteria(guides, sequence):
    mutations = []
    for guide in guides:
        if random.random() < 0.5:  # 50% de posibilidades de una edición exitosa
            start = sequence.find(guide)
            if start != -1:
                end = start + len(guide)
                mutation_type = random.choice(["deletion", "insertion", "substitution"])
                if mutation_type == "deletion":
                    sequence = sequence[:start] + sequence[end:]
                    mutations.append((mutation_type, start, sequence[start:end], ""))
                elif mutation_type == "insertion":
                    insertion_seq = "ATCG"
                    sequence = sequence[:start] + insertion_seq + sequence[start:]
                    mutations.append((mutation_type, start, "", insertion_seq))
                elif mutation_type == "substitution":
                    substitution_seq = "ATCG"
                    sequence = sequence[:start] + substitution_seq + sequence[end:]
                    mutations.append((mutation_type, start, sequence[start:end], substitution_seq))
    return sequence, mutations

# Función para verificar la edición genética
def verify_editing(accession_number, original_sequence, edited_sequence, mutations):
    if original_sequence != edited_sequence:
        print(f"Edición genética verificada para {accession_number}")
        summarize_mutations(mutations)
        graphical_representation(original_sequence, edited_sequence, mutations)
        plot_sequence_differences(original_sequence, edited_sequence, mutations)
        return edited_sequence
    else:
        print(f"No se detectó edición genética para {accession_number}")
        return None

# Función para resumir las mutaciones
def summarize_mutations(mutations):
    print(f"\nTotal de mutaciones: {len(mutations)}")
    count_deletion = sum(1 for m in mutations if m[0] == "deletion")
    count_insertion = sum(1 for m in mutations if m[0] == "insertion")
    count_substitution = sum(1 for m in mutations if m[0] == "substitution")

    print(f"Deleciones: {count_deletion}")
    print(f"Inserciones: {count_insertion}")
    print(f"Sustituciones: {count_substitution}")

# Función para representación gráfica de las ediciones
def graphical_representation(original_seq, edited_seq, mutations):
    print("\nOriginal: " + original_seq[:50] + "..." + original_seq[-50:])
    print("Editada : " + edited_seq[:50] + "..." + edited_seq[-50:])
    for mutation in mutations:
        mutation_type, position, original, new = mutation
        print(f"{'^' * len(original)} {mutation_type.capitalize()} en posición {position}")

# Función para graficar las diferencias de secuencias
def plot_sequence_differences(original_seq, edited_seq, mutations):
    positions = [pos for _, pos, _, _ in mutations]
    mutation_types = [mut_type for mut_type, _, _, _ in mutations]

    plt.figure(figsize=(10, 3))
    plt.scatter(positions, [1] * len(positions), c='red', label='Mutaciones')
    for pos, mut_type in zip(positions, mutation_types):
        plt.annotate(mut_type, (pos, 1), textcoords="offset points", xytext=(0,10), ha='center')

    plt.title('Distribución de Mutaciones en la Secuencia')
    plt.xlabel('Posición en la Secuencia')
    plt.yticks([])
    plt.legend()
    plt.show()

# Ejemplo de uso
accession_number = "NC_016848.1"
with open(accession_number + ".fasta", "r") as file:
    original_sequence = SeqIO.read(file, "fasta").seq

guides = ["GTTACG", "CGTACG"]  # Ejemplo de guías CRISPR
edited_sequence, mutations = transform_bacteria(guides, original_sequence)

edited_seq_record = verify_editing(accession_number, original_sequence, edited_sequence, mutations)
if edited_seq_record:
    SeqIO.write(SeqRecord(Seq(edited_seq_record), id=accession_number), f"{accession_number}_edited.fasta", "fasta")
